In [1]:
import pandas as pd
import numpy as np
import json
import math
# import requests
# import io

In [2]:
# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:10.0) Gecko/20100101 Firefox/10.0'}
# url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
# resp = requests.get(url, headers=headers)
# file_object = io.StringIO(resp.content.decode('utf-8'))

In [3]:
df = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv", 
                     index_col = ['date'],
                     parse_dates = True, 
                     usecols=["date", "iso_code", "location", 
                              "population", "continent",
                              "total_cases", "total_deaths"]).rename(
    columns = {'iso_code': 'country_code', 'location': 'country_name'}
)
df

,country_code,continent,country_name,total_cases,total_deaths,population
date,,,,,,
2020-02-24,AFG,Asia,Afghanistan,1.0,NaN,38928341.0
2020-02-25,AFG,Asia,Afghanistan,1.0,NaN,38928341.0
2020-02-26,AFG,Asia,Afghanistan,1.0,NaN,38928341.0
2020-02-27,AFG,Asia,Afghanistan,1.0,NaN,38928341.0
2020-02-28,AFG,Asia,Afghanistan,1.0,NaN,38928341.0
...,...,...,...,...,...,...
2021-06-17,ZWE,Africa,Zimbabwe,40927.0,1647.0,14862927.0
2021-06-18,ZWE,Africa,Zimbabwe,41335.0,1656.0,14862927.0
2021-06-19,ZWE,Africa,Zimbabwe,41628.0,1666.0,14862927.0


In [4]:
# prepare metadata from World Bank
metadata = pd.read_csv("https://raw.githubusercontent.com/hongtaoh/covid19-data/master/data_sources/metadata/worldbank/country_metadata.csv",
                      usecols=["Country Code", "Region"]).rename(
    columns = {'Country Code': 'country_code', 'Region': 'world_region'}
)

In [5]:
all_dates = pd.date_range(df.index.min(), df.index.max())

In [6]:
def get_fallBehind_place_date_dateframe(df): # input should be df
    fallBehind_place_list = []
    fallBehind_last_date_available_list = []
    fallBehind_last_date_index_list = []
    # With the `groupby` function, each country/region will be a group. 
    # group[1] will be this cuontry/region's data. group[0] will be the `country_code` (in string).
    for group in df.groupby("country_code"):
# for each group, if the last available data isn't the latested in the dataset, 
# then add this group's `country_code` to fallBehind_place_list:
        if group[1].tail(1).index != df.index.max(): 
            fallBehind_place_list.append(group[0])
    for p in fallBehind_place_list:
        fallBehind_last_date_index_list.append(all_dates.get_loc(
            df[df.loc[:, 'country_code']== p].index[-1].strftime("%Y-%m-%d")))
        fallBehind_last_date_available_list.append(
            df[df.loc[:, 'country_code']== p].index[-1].strftime("%Y-%m-%d"))
    d = {'fallBehind_place': fallBehind_place_list, 
         'fallBehind_last_date_available': fallBehind_last_date_available_list,
         'fallBehind_last_date_index': fallBehind_last_date_index_list}
    fallBehind_list = pd.DataFrame(data = d)
    return fallBehind_list

In [29]:
fallBehind_list = get_fallBehind_place_date_dateframe(df)
fallBehind_list

,fallBehind_place,fallBehind_last_date_available,fallBehind_last_date_index
0,ABW,2021-06-21,537
1,AFG,2021-06-21,537
2,AGO,2021-06-21,537
3,AIA,2021-06-18,534
4,ALB,2021-06-21,537
...,...,...,...
224,WSM,2021-06-21,537
225,YEM,2021-06-21,537
226,ZAF,2021-06-21,537
227,ZMB,2021-06-21,537


In [8]:
def extract_cntry_dfs(df): 
    dfs = []
    for group in df.groupby("country_code"):
        cntry_df = (
            group[1].reindex(all_dates, method="ffill")
        )
        cntry_df["total_cases"] = cntry_df["total_cases"]
        cntry_df["total_deaths"] = cntry_df["total_deaths"]
        cntry_df["country_code"] = group[0]
        cntry_df["country_name"] = group[1]["country_name"][-1]
        cntry_df["continent"] = group[1]["continent"][-1]
        cntry_df["population"] = group[1]["population"][-1]
        dfs.append(
            cntry_df
        )
    return dfs

In [9]:
dfs = extract_cntry_dfs(df)

In [10]:
dfs[1].iloc[0]
# dfs[1].iloc[0:1] = [1,'a','c',2,4,'t']

country_code            AFG
continent              Asia
country_name    Afghanistan
total_cases             NaN
total_deaths            NaN
population      3.89283e+07
Name: 2020-01-01 00:00:00, dtype: object

In [11]:
def fill_first_case_death_with_zero(df): # input is dfs
    for i in np.arange(0, len(df)):
        if (df[i].head(1).total_cases.isnull()[0] & df[i].head(1).total_deaths.isnull()[0]):
            df[i][0:1] = [df[i].country_code[-1],
            df[i].continent[-1],
            df[i].country_name[-1],
            0,
            0, 
            df[i].population[-1]
            ]
        if (df[i].head(1).total_cases.isnull()[0] & df[i].head(1).total_deaths.notnull()[0]):
            df[i][0:1] = [df[i].country_code[-1],
            df[i].continent[-1],
            df[i].country_name[-1],
            0,
            df[i].total_deaths[0],
            df[i].population[-1]
            ]
        if (df[i].head(1).total_cases.notnull()[0] & df[i].head(1).total_deaths.isnull()[0]):
            df[i][0:1] = [df[i].country_code[-1],
            df[i].continent[-1],
            df[i].country_name[-1],
            df[i].total_cases[0],
            0,
            df[i].population[-1]
            ]
    return df # output is the dfs with first case & death conditionally filled with zero. 
              # Later, I name this output to be "dfs_first_zero_filled" 

In [12]:
dfs_first_zero_filled = fill_first_case_death_with_zero(dfs)
type(dfs_first_zero_filled[1])

pandas.core.frame.DataFrame

In [13]:
concat_df = pd.concat(dfs_first_zero_filled).fillna(method="ffill").reset_index()
concat_df.loc[:,"index"]

0        2020-01-01
1        2020-01-02
2        2020-01-03
3        2020-01-04
4        2020-01-05
            ...    
123965   2021-06-18
123966   2021-06-19
123967   2021-06-20
123968   2021-06-21
123969   2021-06-22
Name: index, Length: 123970, dtype: datetime64[ns]

In [14]:
concat_df = pd.concat(dfs_first_zero_filled).fillna(method="ffill").reset_index().rename(
        columns={"index": "date"})
    #To change the original country codes of "KOS" and World to match metadata from WB:
concat_df.loc[(concat_df.country_code == "OWID_KOS"), ('country_code')] = "XKX"
concat_df.loc[(concat_df.country_code == "OWID_WRL"), ('country_code')] = "WLD"
    # To get the column of "world_region" in concat_df by merging with WB metadata
left_join_df = pd.merge(concat_df, metadata, on = "country_code", how = "left")
# df[df['name column'].isnull()]
left_join_df.loc[left_join_df.world_region.isnull()].country_code.unique()

array(['AIA', 'BES', 'COK', 'FLK', 'GGY', 'JEY', 'MSR', 'OWID_AFR',
       'OWID_ASI', 'OWID_CYN', 'OWID_EUN', 'OWID_EUR', 'OWID_INT',
       'OWID_NAM', 'OWID_OCE', 'OWID_SAM', 'WLD', 'PCN', 'SHN', 'TWN',
       'VAT', 'WLF'], dtype=object)

In [15]:
def merge_with_meta(df): #input should be dfs_first_zero_filled
    concat_df = pd.concat(df).fillna(method="ffill").reset_index().rename(
        columns={"index": "date"})
    #To change the original country codes of "KOS" and World to match metadata from WB:
    concat_df.loc[(concat_df.country_code == "OWID_KOS"), ('country_code')] = "XKX"
    concat_df.loc[(concat_df.country_code == "OWID_WRL"), ('country_code')] = "WLD"
    # To get the column of "world_region" in concat_df by merging with WB metadata
    left_join_df = pd.merge(concat_df, metadata, on = "country_code", how = "left")
    left_join_df.loc[:,'date'] = left_join_df.loc[:,'date'].dt.strftime('%Y-%m-%d')
    left_join_df.loc[(left_join_df.country_code == "AIA"), ('world_region')] = "Latin America & Caribbean"
    left_join_df.loc[(left_join_df.country_code == "BES"), ('world_region')] = "Latin America & Caribbean"
    left_join_df.loc[(left_join_df.country_code == "ESH"), ('world_region')] = "Middle East & North Africa"
    left_join_df.loc[(left_join_df.country_code == "FLK"), ('world_region')] = "Latin America & Caribbean"
    left_join_df.loc[(left_join_df.country_code == "GGY"), ('world_region')] = "Europe & Central Asia"
    left_join_df.loc[(left_join_df.country_code == "JEY"), ('world_region')] = "Europe & Central Asia"
    left_join_df.loc[(left_join_df.country_code == "MSR"), ('world_region')] = "Latin America & Caribbean"
    left_join_df.loc[(left_join_df.country_code == "TWN"), ('world_region')] = "East Asia & Pacific"
    left_join_df.loc[(left_join_df.country_code == "VAT"), ('world_region')] = "Europe & Central Asia"
    left_join_df.loc[(left_join_df.country_code == "WLF"), ('world_region')] = "East Asia & Pacific"
    left_join_df.loc[(left_join_df.country_code == "WLD"), ('world_region')] = "World"
    return left_join_df

In [16]:
left_join_df = merge_with_meta(dfs_first_zero_filled)

In [17]:
# In the following step, I converted 
def fallBehind_filled_to_null (df): # input should be left_join_df
    left_join_copy_group1_with_nan = []
    for group in df.groupby('country_code'):
        for i in np.arange(0, len(fallBehind_list)):
            if group[1].tail(1).country_code.iloc[0] == fallBehind_list.iloc[i, 0]:
                group[1].tail(len(all_dates) - 1 - fallBehind_list.iloc[i, 2]).total_cases = np.nan
                group[1].tail(len(all_dates) - 1 - fallBehind_list.iloc[i, 2]).total_deaths = np.nan
        left_join_copy_group1_with_nan.append(group[1])
    left_join_copy_group1_with_nan_concated = pd.concat(left_join_copy_group1_with_nan)
    left_join_copy_group1_concated_with_null = left_join_copy_group1_with_nan_concated
    left_join_copy_group1_concated_with_null.replace(np.nan, 'null', inplace=True)
    return left_join_copy_group1_concated_with_null 
# Later, I'll name the output to be fallBehind_with_nan_concated

In [18]:
# fallBehind_with_nan_concated = fallBehind_filled_to_nan(left_join_df)

In [19]:
fallBehind_with_null = fallBehind_filled_to_null(left_join_df)

/opt/anaconda3/envs/dviz/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [20]:
# fallBehind_with_null.replace(np.nan, 'null', inplace=True)

In [21]:
# fallBehind_with_null[fallBehind_with_null.loc[:, 'country_code'] == 'BEL'].tail(10)

In [22]:
# fallBehind_nan_changed[fallBehind_nan_changed.isnull().any(axis=1)]

In [23]:
fallBehind_with_null[fallBehind_with_null.loc[:, 'country_code'] == 'SWE'].tail(10)

,date,country_code,continent,country_name,total_cases,total_deaths,population,world_region
105095,2021-06-13,SWE,Europe,Sweden,1.08346e+06,14574,10099270.0,Europe & Central Asia
105096,2021-06-14,SWE,Europe,Sweden,1.08346e+06,14574,10099270.0,Europe & Central Asia
105097,2021-06-15,SWE,Europe,Sweden,1.08464e+06,14574,10099270.0,Europe & Central Asia
105098,2021-06-16,SWE,Europe,Sweden,1.08464e+06,14574,10099270.0,Europe & Central Asia
105099,2021-06-17,SWE,Europe,Sweden,1.08464e+06,14574,10099270.0,Europe & Central Asia
105100,2021-06-18,SWE,Europe,Sweden,1.08464e+06,14574,10099270.0,Europe & Central Asia
105101,2021-06-19,SWE,Europe,Sweden,1.08464e+06,14574,10099270.0,Europe & Central Asia
105102,2021-06-20,SWE,Europe,Sweden,1.08464e+06,14574,10099270.0,Europe & Central Asia
105103,2021-06-21,SWE,Europe,Sweden,1.08464e+06,14574,10099270.0,Europe & Central Asia
105104,2021-06-22,SWE,Europe,Sweden,null,null,10099270.0,Europe & Central Asia


In [24]:
def prepare_data_structure(df, gby="country_code"): # input should be fallBehind_with_null
    data = []
    for g in df.groupby([gby]):
        code = g[0]
        cntry_df = g[1]
        try:
            country_data = {
                "country_code": code,
                "country_name": cntry_df.loc[:,"country_name"].iloc[0],
                "population": cntry_df.loc[:,"population"].iloc[0],
                "region": cntry_df.loc[:,"world_region"].iloc[0],
                "confirmed": list(zip(cntry_df.date, cntry_df.total_cases)),
                "deaths": list(zip(cntry_df.date, cntry_df.total_deaths)),
            }
            data.append(country_data)
        except KeyError:
            print("metadata doesn't exist for: ", code)
            continue
    return data

In [25]:
data = prepare_data_structure(fallBehind_with_null)

In [26]:
open("../output/cntry_stat_owid.json", "w").write(json.dumps(data, separators=(",", ":")))

5258357